In [1]:
#import libraries
from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
#from __future__ import division
from sklearn.cluster import KMeans


import plotly.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go

import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split

import xgboost as xgb

#initate plotly
pyoff.init_notebook_mode()


In [6]:
#read data from csv and redo the data work we done before
tx_data = pd.read_csv('OrderDataset.csv')
tx_data['InvoiceDate'] = pd.to_datetime(tx_data['InvoiceDate'])
#tx_uk = tx_data.query("Country=='United Kingdom'").reset_index(drop=True)


In [8]:
#create 3m and 6m dataframes
tx_3m = tx_data[(tx_data.InvoiceDate < date(2017,6,1)) & (tx_data.InvoiceDate >= date(2017,3,1))].reset_index(drop=True)
tx_6m = tx_data[(tx_data.InvoiceDate >= date(2017,6,1)) & (tx_data.InvoiceDate < date(2017,12,1))].reset_index(drop=True)

C:\anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning:

Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.

C:\anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning:

Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.



In [9]:
#create tx_user for assigning clustering
tx_user = pd.DataFrame(tx_3m['CustomerID'].unique())
tx_user.columns = ['CustomerID']

In [10]:
#order cluster method
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final


In [11]:
#calculate recency score
tx_max_purchase = tx_3m.groupby('CustomerID').InvoiceDate.max().reset_index()
tx_max_purchase.columns = ['CustomerID','MaxPurchaseDate']
tx_max_purchase['Recency'] = (tx_max_purchase['MaxPurchaseDate'].max() - tx_max_purchase['MaxPurchaseDate']).dt.days
tx_user = pd.merge(tx_user, tx_max_purchase[['CustomerID','Recency']], on='CustomerID')


In [12]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(tx_user[['Recency']])
tx_user['RecencyCluster'] = kmeans.predict(tx_user[['Recency']])

In [13]:
tx_user = order_cluster('RecencyCluster', 'Recency',tx_user,False)

In [14]:
#calcuate frequency score
tx_frequency = tx_3m.groupby('CustomerID').InvoiceDate.count().reset_index()
tx_frequency.columns = ['CustomerID','Frequency']
tx_user = pd.merge(tx_user, tx_frequency, on='CustomerID')

In [15]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(tx_user[['Frequency']])
tx_user['FrequencyCluster'] = kmeans.predict(tx_user[['Frequency']])

tx_user = order_cluster('FrequencyCluster', 'Frequency',tx_user,True)

In [16]:
#calcuate revenue score
tx_3m['Revenue'] = tx_3m['UnitPrice'] * tx_3m['Quantity']
tx_revenue = tx_3m.groupby('CustomerID').Revenue.sum().reset_index()
tx_user = pd.merge(tx_user, tx_revenue, on='CustomerID')

kmeans = KMeans(n_clusters=4)
kmeans.fit(tx_user[['Revenue']])
tx_user['RevenueCluster'] = kmeans.predict(tx_user[['Revenue']])
tx_user = order_cluster('RevenueCluster', 'Revenue',tx_user,True)

In [17]:
#overall scoring
tx_user['OverallScore'] = tx_user['RecencyCluster'] + tx_user['FrequencyCluster'] + tx_user['RevenueCluster']
tx_user['Segment'] = 'Low-Value'
tx_user.loc[tx_user['OverallScore']>2,'Segment'] = 'Mid-Value' 
tx_user.loc[tx_user['OverallScore']>4,'Segment'] = 'High-Value' 

In [18]:
tx_user.head()

CustomerID  Recency  RecencyCluster  Frequency  FrequencyCluster  Revenue  \
0     14620.0       12               3         30                 0   393.28   
1     15194.0        6               3         64                 0  1439.02   
2     18044.0        5               3         57                 0   808.96   
3     18075.0       12               3         35                 0   638.12   
4     15241.0        0               3         64                 0   947.55   

   RevenueCluster  OverallScore    Segment  
0               0             3  Mid-Value  
1               0             3  Mid-Value  
2               0             3  Mid-Value  
3               0             3  Mid-Value  
4               0             3  Mid-Value

In [19]:
#calculate revenue and create a new dataframe for it
tx_6m['Revenue'] = tx_6m['UnitPrice'] * tx_6m['Quantity']
tx_user_6m = tx_6m.groupby('CustomerID')['Revenue'].sum().reset_index()
tx_user_6m.columns = ['CustomerID','m6_Revenue']

In [20]:
#plot LTV histogram
plot_data = [
    go.Histogram(
        x=tx_user_6m.query('m6_Revenue < 10000')['m6_Revenue']
    )
]

plot_layout = go.Layout(
        title='6m Revenue'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [32]:
from __future__ import division
tx_graph = tx_merge.query("m6_Revenue < 30000")

plot_data = [
    go.Scatter(
        x=tx_graph.query("Segment == 'Low-Value'")['OverallScore'],
        y=tx_graph.query("Segment == 'Low-Value'")['m6_Revenue'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'Mid-Value'")['OverallScore'],
        y=tx_graph.query("Segment == 'Mid-Value'")['m6_Revenue'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'High-Value'")['OverallScore'],
        y=tx_graph.query("Segment == 'High-Value'")['m6_Revenue'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "6m LTV"},
        xaxis= {'title': "RFM Score"},
        title='LTV'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [31]:
#mergin 3m and 6m dataframe
tx_merge = pd.merge(tx_user, tx_user_6m, on='CustomerID', how='left')

#convert NaN values to 0
tx_merge = tx_merge.fillna(0)

#show correlation between overall RFM score and LTV
tx_graph = tx_merge.query("m6_Revenue < 30000")

plot_data = [
    go.Scatter(
        x=tx_graph.query("Segment == 'Low-Value'")['OverallScore'],
        y=tx_graph.query("Segment == 'Low-Value'")['m6_Revenue'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'Mid-Value'")['OverallScore'],
        y=tx_graph.query("Segment == 'Mid-Value'")['m6_Revenue'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'High-Value'")['OverallScore'],
        y=tx_graph.query("Segment == 'High-Value'")['m6_Revenue'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "RFM Score"},
        xaxis= {'title': "6m Revenue"},
        title='LTV'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [33]:
#remove outliers
tx_merge = tx_merge[tx_merge['m6_Revenue']<tx_merge['m6_Revenue'].quantile(0.99)]


#creating 3 clusters
kmeans = KMeans(n_clusters=3)
kmeans.fit(tx_merge[['m6_Revenue']])
tx_merge['LTVCluster'] = kmeans.predict(tx_merge[['m6_Revenue']])

#order cluster number based on LTV
tx_merge = order_cluster('LTVCluster', 'm6_Revenue',tx_merge,True)

#creatinga new cluster dataframe
tx_cluster = tx_merge.copy()

#see details of the clusters
tx_cluster.groupby('LTVCluster')['m6_Revenue'].describe()

count          mean          std      min        25%        50%  \
LTVCluster                                                                     
0           1724.0    572.224670   610.253082  -609.40     0.0000    370.805   
1            256.0   3815.228711  1430.791714  2199.38  2686.2725   3439.235   
2             26.0  12377.135000  3210.865481  8439.09  9607.3225  11861.385   

                   75%       max  
LTVCluster                        
0             951.4375   2195.62  
1            4431.7575   7946.90  
2           13745.0575  19289.53

In [34]:
#convert categorical columns to numerical
tx_class = pd.get_dummies(tx_cluster)

#calculate and show correlations
corr_matrix = tx_class.corr()
corr_matrix['LTVCluster'].sort_values(ascending=False)

#create X and y, X will be feature set and y is the label - LTV
X = tx_class.drop(['LTVCluster','m6_Revenue'],axis=1)
y = tx_class['LTVCluster']

#split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=56)

In [35]:
tx_cluster.head()

CustomerID  Recency  RecencyCluster  Frequency  FrequencyCluster  Revenue  \
0     14620.0       12               3         30                 0   393.28   
1     18044.0        5               3         57                 0   808.96   
2     18075.0       12               3         35                 0   638.12   
3     15241.0        0               3         64                 0   947.55   
4     12679.0       13               3         47                 0   869.13   

   RevenueCluster  OverallScore    Segment  m6_Revenue  LTVCluster  
0               0             3  Mid-Value        0.00           0  
1               0             3  Mid-Value      991.54           0  
2               0             3  Mid-Value     1322.75           0  
3               0             3  Mid-Value      791.04           0  
4               0             3  Mid-Value      189.54           0

In [36]:
tx_class.head()

CustomerID  Recency  RecencyCluster  Frequency  FrequencyCluster  Revenue  \
0     14620.0       12               3         30                 0   393.28   
1     18044.0        5               3         57                 0   808.96   
2     18075.0       12               3         35                 0   638.12   
3     15241.0        0               3         64                 0   947.55   
4     12679.0       13               3         47                 0   869.13   

   RevenueCluster  OverallScore  m6_Revenue  LTVCluster  Segment_High-Value  \
0               0             3        0.00           0                   0   
1               0             3      991.54           0                   0   
2               0             3     1322.75           0                   0   
3               0             3      791.04           0                   0   
4               0             3      189.54           0                   0   

   Segment_Low-Value  Segment_Mid-Value  
0                  0                  1  
1                  0                  1  
2                  0                  1  
3                  0                  1  
4                  0                  1

In [37]:
corr_matrix = tx_class.corr()
corr_matrix['LTVCluster'].sort_values(ascending = False)

LTVCluster            1.000000
m6_Revenue            0.852419
Revenue               0.595305
RevenueCluster        0.512696
Segment_High-Value    0.407606
Frequency             0.363203
OverallScore          0.363057
FrequencyCluster      0.357739
RecencyCluster        0.208669
Segment_Mid-Value     0.116876
CustomerID           -0.090203
Recency              -0.206322
Segment_Low-Value    -0.256989
Name: LTVCluster, dtype: float64

In [38]:
#XGBoost Multiclassification Model
ltv_xgb_model = xgb.XGBClassifier(max_depth=5, learning_rate=0.1,objective= 'multi:softprob',n_jobs=-1).fit(X_train, y_train)

print('Accuracy of XGB classifier on training set: {:.2f}'
       .format(ltv_xgb_model.score(X_train, y_train)))
print('Accuracy of XGB classifier on test set: {:.2f}'
       .format(ltv_xgb_model.score(X_test[X_train.columns], y_test)))

y_pred = ltv_xgb_model.predict(X_test)
print(classification_report(y_test, y_pred))

Accuracy of XGB classifier on training set: 0.95
Accuracy of XGB classifier on test set: 0.89
              precision    recall  f1-score   support

           0       0.93      0.97      0.95        87
           1       0.62      0.38      0.48        13
           2       0.33      1.00      0.50         1

   micro avg       0.89      0.89      0.89       101
   macro avg       0.63      0.78      0.64       101
weighted avg       0.89      0.89      0.88       101

